In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py

In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.environ['LANGCHAIN_TRACING_V2'] = 'true'
LANGCHAIN_ENDPOINT = os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

docs = FireCrawlLoader (api_key = LANGCHAIN_API_KEY, url="https://www.tokyotechies.com/about-us", mode="scrape").load()

#Split documents
docs_list = [item for sublist in docs for item in sublist]

#Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250, chunk_overlap = 0
)
doc_splits = text_splitter.split_documents(docs_list)

#Filter out complex metadata
filtered_docs = []
for doc in doc_splits:
    if isinstance(doc, Document) and hasattri(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))

#Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=filtered_docs,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()


